## User Story:
The IT department of _**Plain Corp**_ has implemented a custom-built _Computer Management Solution (CMS)_ which comes with a feature that reports information about computers in their environment such as metadata, installed programs and logins. The current problem faced by the IT team is that the reports created by CMS do not answer all the questions they have about the computers in their environment. Luckily, they figured out that the CMS stores its raw data in a SQLite (v3.45.0)  _database (DB)_ file and that they could write custom SQL queries to report the information not provided within the CMS reports.

## Goal:
Your job will be to update this Jupyter Notebook with the SQL queries needed to meet the IT team’s reporting needs. It is recommended to use a Database IDE to help with query writing and data exploration.

## Directions:
Download the attached zip file which contains all the files you need to complete this assignment. Below is a description of each of the three files within the attached zip file:
- **cms.sqlite**: The database you must query from to complete this assignment.
- **cms_data_dictionary.txt**: A data dictionary describing the tables within this database.
- **GA Mission - Data Analyst.ipynb**: The Jupyter Notebook file you must update to complete the assignment. Rules to abide by while completing this assignment are listed within this Jupyter Notebook.

## Our Expectations
This assignment <ins>**should take you no longer than 2 hours**</ins> to complete. Make sure your outputs are visible in a web browser when uploaded to a public repository and that they are presented in a way that is understandable by someone with some technical experience but no programming experience. Extra rules to abide by are described in the included Juptyter Notebook file.

We would prefer that you share the code with us by pushing it to a public repository in the hosting provider of your choosing. If you do not want to upload it to a public repository then you can zip up the repository folder on your machine and send it to us via email. If you go that route, please make sure that the .git folder is included.

We recommend  that you use a Database IDE (e.g., DBeaver, Datagrip, etc.) to assist with query writing and data exploration. If you have any questions, please contact us and we will do our best to clarify anything that is unclear.

## Rules:
- The code should be stored in a git repository.
- With the exception of ***Pandas***, you can use any Python Packages you like.
- Do not edit the data in the **cms.sqlite** DB file.
- Your query results must be output in a _Pandas DataFrame (PD)_.
- Questions 4 and 5 are written answer questions where no code or SQL is necessary.
- Question 6 is a written answer question that may require code or SQL depending on your answer. This question can be thought of as a chance to share any observations you've made or perform a light EDA on the data. There are no right or wrong answers for this question.

# Questions

In [15]:
import pandas as pd
import sqlite3 as db

conn = db.connect("/kaggle/input/galacticsolutionsmission/cms.sqlite")
cur = conn.cursor()

 <font size='5'>1. Count how many Admin and Regular computers are in the database.</font>

In [16]:
query="SELECT COUNT(computer_type) AS CountAdminRegular FROM computers WHERE computer_type = 'Admin' OR 'Regular'"
comp_count = pd.read_sql(query,conn)
print(comp_count)

   CountAdminRegular
0                  8


<font size='5'>2a. Identify all non-admin computers with the programs **Bitwarden**, **Quickpass** or **1password** installed, please note that the listed program names may not match exactly to the program names in the database. Be sure to include the computer names, operating system, program names and program version in the PD output.</font>

In [17]:
query="SELECT DISTINCT(c.computer_name), c.operating_system,ip.program_name,ip.program_version FROM computers c INNER JOIN installed_programs ip on c.computer_name = ip.computer_name WHERE (ip.program_name = 'Quickpass' OR ip.program_name = '/Applications/Bitwarden.app' OR ip.program_name = '1password') AND (c.computer_type IS NOT 'Admin')"
non_admin = pd.read_sql(query,conn)

print(non_admin)

       computer_name                            operating_system  \
0    DESKTOP-O23A1M7                    Microsoft Windows 11 Pro   
1    LAPTOP-55433J3D               Microsoft Windows 11 Business   
2     PLAINCORP-1012                    Microsoft Windows 11 Pro   
3     PLAINCORP-1014               Microsoft Windows 11 Business   
4     PLAINCORP-1014               Microsoft Windows 11 Business   
5    PLAINCORP-DRAKE                    Microsoft Windows 11 Pro   
6   PLAINCORP-WILL-P               Microsoft Windows 11 Business   
7   Testers-Mac-mini                         macOS 14.0 (23A344)   
8       Testers-Mini               macOS 13.3.1 (a) (22E772610a)   
9     WINDEV2303EVAL  Microsoft Windows 11 Enterprise Evaluation   
10   ZPPLAINCORP-MAC                          macOS 13.2 (22D49)   

                   program_name program_version  
0                     Quickpass   15.0.21225.01  
1                     Quickpass   15.0.21225.01  
2                     1password  

<font size='5'>2b. Report the latest version of each program highlighted in question **2a** and how many non-admin computers have said programs installed.</font>

In [18]:
query="SELECT COUNT(c.computer_name) AS NonAdminComputerCount, ip.program_name, ip.program_version AS latest_version FROM computers c INNER JOIN ( SELECT ip.computer_name, ip.program_name, MAX(ip.program_version) AS program_version FROM installed_programs ip WHERE (ip.program_name = 'Quickpass' OR ip.program_name = '/Applications/Bitwarden.app' OR ip.program_name = '1password') GROUP BY ip.computer_name, ip.program_name ) ip on c.computer_name = ip.computer_name WHERE c.computer_type IS NOT 'Admin' GROUP BY ip.program_name ORDER BY latest_version DESC;"
non_admin_latest_program = pd.read_sql(query,conn)

print(non_admin_latest_program)

   NonAdminComputerCount                 program_name latest_version
0                      3  /Applications/Bitwarden.app       2023.1.1
1                      2                    1password          2.7.4
2                      6                    Quickpass  15.0.21225.01


<font size='5'>2c. Report each program highlighted in question **2a** and the ratio of which computers out of all computers have said programs installed. In one column express the ratio in the format of “**X** out of **Y**“ computers, in another column express the ration as a percentage rounded to the 2nd decimal point. Hint: To calculate the percentage, you will have to cast one of the numbers to a decimal by multiplying them by **1.0**.</font>

I ran out of time for this one but I do have thought solutions. 

similar subquery as part 2a, I don't know how to do the format of "X out of Y" but I did know how to do the column expression. I was also making this a little harder by trying ot get the ratio of computers that had the latest software. I now realize it was asking for installed programs regardless of program version. 

<font size='5'>3a. Report which admin computers have logins on or later than May 2023, have **Python** installed on their computers and whose IP addresses are not in the IP whitelist table. Be sure to include the computer name, IP address and login timestamp; order the results by most recent date and computer name.</font>

Ran out of Time to solve this to my standards but here's my thought process: 

this is a 4 join query, I would probably use a query with combination of CTEs becuase it makes it easier to read as a developer, and also easy enough to follow as a business user. so it would be a 3 CTE + Outermost Query to complete this one with the appropriate fields and filters. I would filter the IP addresses in the CTE for IP whitelist, but leave the other 2 filters at the end since the tables are bigger. 

<font size='5'>3b. Below are the requirements for the modifications you should make to the PD output from **3a**, you cannot change the SQL query to meet these requirements:</font>
- <font size='5'>Replace the 2nd and 4th octets of the IP address with **\*** (e.g., 10.\*.0.\*). Your solution must use regex.</font>
- <font size='5'>Format the login timestamp as the date only (e.g., 01/20/2024)</font>


Ran out of Time, but thought process

I would have to spend some time workign on the regex but I imagine it would look like this
10(match dot)(match star)(match dot)0(match dot)(match star)

I would use pandas to_datetime with format (moth,day,year)

<font size='5'>4. Bob from the IT department noticed a SQL query was starting to get slow and since he knew a little bit of SQL from college, he decided to fix the query himself. The first thing Bob did was to reduce the number of unnecessary joins in the query, but the new query was still slow. Was Bob’s initial approach the correct one? If so, explain Bob's next steps? If not, explain what Bob should've done instead?</font>

I'm not sure if Bob's approach was correct, although it was a good step by itself. Normally you do want to reduce unnecessary joins.

Normally in my experience this is what I would have Bob look into. 

One of the things I know from experience is selecting the necessary comlumns across all joins is crucial. Specially when dealing with multiple tables of over 500k records if you're just selecting * it takes a big hit. Select necessary fields across all joins and that should help out a bit. 

Overuse of Group BY is a problem too at a big scale. I usually try not to group until the very end of my query (outermost query) that's because group by is taxing and if you're grouping by in all the sub queries it will cost you performance over time. 


<font size='5'>5. To improve the CMS’s backend and reporting capabilities, the IT department wants to provide feedback to the contractor who built the CMS solution and wants your input. How would you improve or change the data model? If you decide to create a data diagram you must save it as a PDF or image file in the same folder as this Jupyter Notebook.</font>

Ran out of time, but some of my improvement suggestions would probably be in the path of my answers to question 6.

<font size='5'>6. Did you notice anything interesting or anything wrong with the data? If so, please describe what you found and if possible, provide any related evidence.</font>

List of Observations by Table:

Computer:
- Handful of nulls in last scan of computers, hard to tell if the computers are test computers or client(business) computers, maybe worth adding a couple fields/tables relating computers to clients and or behavioral stuff (test/actual business use). 
- inconsistency in patter of computer naming. As an analyst I expect this, but if say a data architect were to see this they would want a framework to define these fields better. 


Login Records:
- assuming all logins are successful, from a monitoring perspective it would be nice to know if these records were attempts or actual logins, another comlumn would be nice
- I would also add a user foreign key just to make it easier to track. for reporting or analysis purposes it's just faster, I don't think there would be too much technical processing cost to have that relative to the business value it saves later down at my level. 

Installed Programs: 
- I would definitely try to get a better mapping for the program names, I understand why they are the way the are, but that pathing and variation in naming really causes a hassle down the road, it's fine for 'back-end' side if you will, but if more business people come in and consume this they're gonna have a much harder time. In the spirit of my favorite phrase, slow is smooth, smooth is fast. My thought is always beign careful in the beginning and making something that creates speed in the downstream. 

IP_Whitelist:
- I would like a network submask field, and maybe IPV4/IPV6 classification too. Again this is one of those that is like if you look at it you know, and to some degree we always expect/assume people will know. But classification and orderliness is really the name of the game. 

